In [2]:
import pandas as pd

In [1]:
pip install lxml


The following command must be run outside of the IPython shell:

    $ pip install lxml

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
import requests
import pandas as pd
#read website date into a dataframe
Toronto_data=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
Toronto_data=Toronto_data[0]

# stack all the data into a single column
Toronto_datanew = pd.Series(Toronto_data.values.ravel('F'))

Toronto_code = Toronto_datanew.to_frame()
Toronto_code.columns = ['A']

#Extract the PostalCode and Borough 
Toronto_code['PostalCode']=Toronto_code['A'].astype(str).str[0:3]
Toronto_code['Borough']=Toronto_code['A'].astype(str).str[3:]

# delete all records with Borough as 'Not Assigned'
Toronto_code = Toronto_code[Toronto_code.Borough != 'Not assigned']
Toronto_code = Toronto_code[['PostalCode','Borough']]

#Extract the data for Neighborhood 
import re
Toronto_code['Neighborhood'] = Toronto_code['Borough'].apply(lambda x: re.findall('\((.*?)\)',x))
df=Toronto_code['Borough'].str.split('(', expand=True)
df.columns = ['A','B','C']
Toronto_code['Borough'] = df['A'] 
Toronto_code = Toronto_code[['PostalCode','Borough','Neighborhood']]
Toronto_code['Neighborhood'] = Toronto_code['Neighborhood'].str.get(0)
#Sort the data by POstal Code and Borough
Toronto_code = Toronto_code.sort_values(by =['PostalCode','Borough'])

#Check for duplicate entrees for Postal Code
Toronto_code1 = Toronto_code[Toronto_code.Neighborhood != 'Not assigned']

#Total Records is equal to Total unique records for column PostalCode

# Merge the Neighborhood Data if PostalCode and Borough are Same

NumofRows = len(Toronto_code.index)
count = 0
nextcount = 1
while count!=NumofRows:
    if nextcount == NumofRows:
        break
    if ((Toronto_code['PostalCode'].iloc[count] == Toronto_code['PostalCode'].iloc[nextcount]) & (Toronto_code['PostalCode'].iloc[count] == Toronto_code['Borough'].iloc[nextcount])):
        Toronto_code['Neighborhood'].iloc[count] = (Toronto_code['PostalCode'].iloc[count] + ' , ' + Toronto_code['PostalCode'].iloc[nextcount])
        Toronto_code.drop(df.index[[nextcount]])
    count = count + 1
    nextcount = nextcount +1


In [4]:
Toronto_code.shape

(103, 3)

In [5]:
#extract the Longitute and latitude data
Toronto_geodata=pd.read_csv("http://cocl.us/Geospatial_data")

# Sort the data by Postal Code
Toronto_geodata = Toronto_geodata.sort_values(by =['Postal Code'])
Toronto_geodata = Toronto_geodata.rename(columns={"Postal Code": "PostalCode"})

# Add the longitude and latitude date to Original data frame
Toronto_code = Toronto_code.merge(Toronto_geodata, on="PostalCode", how = 'inner')
